In [1]:
from pathlib import Path

import pandas as pd
from neptoon.quality_control import (
    QualityAssessmentFlagBuilder,
    QualityCheck,
    QATarget,
    QAMethod,
)
from neptoon.corrections import (
    CorrectionType,
    CorrectionTheory,
)
from neptoon.calibration import CalibrationConfiguration
from neptoon.io.read import (
    ConfigurationManager
    
)
from neptoon.columns import ColumnInfo
from neptoon.workflow import ProcessWithYaml

In [2]:
# DataAuditLog.create()

In [3]:
config = ConfigurationManager()
station_config_path = (
    Path.cwd() / "configuration_files" / "FSC001_station.yaml"
)
processing_config_path = (
    Path.cwd() / "configuration_files" / "v1_processing_method.yaml"
)

config.load_configuration(
    file_path=station_config_path,
)
config.load_configuration(
    file_path=processing_config_path,
)

yaml_processor = ProcessWithYaml(configuration_object=config)

## OPTION 1:
data_hub = yaml_processor.create_data_hub()

In [ ]:
data_hub.crns_data_frame

In [5]:
calib_df = pd.read_csv(
    Path.cwd() / "example_data" / "FSCD001_calibration.csv"
)
data_hub.calibration_samples_data = calib_df

The `attach_nmdb_data()` method has defaults. Running the below code, without selecting values, will work too. 


```python
data_hub.attach_nmdb_data()
```



In [ ]:
data_hub.attach_nmdb_data(
    station = "JUNG",
    new_column_name = "incoming_neutron_intensity",
    resolution = "60",
    nmdb_table = "revori"
    )
data_hub.crns_data_frame

In [ ]:
data_hub.prepare_static_values()
data_hub.crns_data_frame

You can use your IDE (e.g., in VSCode) to check which methods are available using the 

In [ ]:
from neptoon.quality_control import WhatParamsDoINeed

WhatParamsDoINeed(QAMethod.SPIKE_UNILOF)

The parameters are optional and uses defaults, I'll stick with these.

In [ ]:
qa_flags = QualityAssessmentFlagBuilder()
qa_flags.add_check(
    
    QualityCheck(
        target=QATarget.RELATIVE_HUMIDITY,
        method=QAMethod.RANGE_CHECK,
        parameters={
            "min":0,
            "max":100
                }),

    QualityCheck(
        target=QATarget.RAW_EPI_NEUTRONS,
        method=QAMethod.SPIKE_UNILOF,
    )
)

data_hub.add_quality_flags(custom_flags=qa_flags)
data_hub.apply_quality_flags()
data_hub.flags_data_frame


In [ ]:
data_hub.smooth_data(
    column_to_smooth=str()
)

In [ ]:
data_hub.select_correction(
    correction_type=CorrectionType.INCOMING_INTENSITY,
    correction_theory=CorrectionTheory.HAWDON_2014,
)
data_hub.select_correction(
    correction_type=CorrectionType.HUMIDITY,
    correction_theory=CorrectionTheory.ROSOLEM_2013
)

data_hub.select_correction(
    correction_type=CorrectionType.PRESSURE,
)

# data_hub.select_correction(
#     correction_type=CorrectionType.ABOVE_GROUND_BIOMASS
# )

data_hub.correct_neutrons()
data_hub.crns_data_frame

We've pulled some calibration data that doesn't exactly follow the expected naming convention in neptoon. We can change this in the `CalibrationConfiguration` by updating the expected column names for key columns. 

In [ ]:
calibration_config = CalibrationConfiguration(
    date_time_column_name='DateTime_utc',
    profile_id_column='Profile_ID',
    distance_column='Distance_to_CRNS_m',
    sample_depth_column='Profile_Depth_cm',
    soil_moisture_gravimetric_column='SoilMoisture_g_g',
    bulk_density_of_sample_column='DryBulkDensity_g_cm3',
    soil_organic_carbon_column='SoilOrganicCarbon_g_g',
    lattice_water_column='LatticeWater_g_g',
)

data_hub.calibrate_station(config=calibration_config)
data_hub.calibrator.return_calibration_results_data_frame()

In [ ]:
data_hub.produce_soil_moisture_estimates()
data_hub.crns_data_frame

In [ ]:
figs = data_hub.create_figures(show_figures=True)

In [ ]:
data_hub.figure_creator.temp_handler.get_figures()

In [15]:
df = data_hub.crns_data_frame


In [14]:
data_hub.save_data()